In [9]:
import pandas as pd

# Datensätze einlesen
apartments = pd.read_csv("original_apartment_data_analytics_hs24_with_lat_lon.csv")
minergie = pd.read_csv("ogd91_minergiegebaeude_pro_gemeinde.csv")

# Zusammenführen anhand BFS-Nummer
merged_data = apartments.merge(minergie, left_on='bfs_number', right_on='BfsNumber', how='left')

# Minergie-Feature erstellen
merged_data['minergie_total'] = (
    merged_data['Minergie'] +
    merged_data['Minergie_Eco'] +
    merged_data['Minergie_A'] +
    merged_data['Minergie_A_Eco'] +
    merged_data['Minergie_P'] +
    merged_data['Minergie_P_Eco']
)

# Prüfen auf fehlende Werte und ggf. füllen
merged_data['minergie_total'] = merged_data['minergie_total'].fillna(merged_data['minergie_total'].median())


In [10]:
# Feature-Auswahl
features = merged_data[['area', 'rooms', 'lat', 'lon', 'minergie_total']]
target = merged_data['price']


In [11]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Trainings- und Testdatensatz erstellen
X_train, X_test, y_train, y_test = train_test_split(
    features, target, test_size=0.2, random_state=42
)

model = RandomForestRegressor(
    n_estimators=10,    # sehr wenige Bäume
    max_depth=8,        # geringe Tiefe
    min_samples_split=20,  # weniger Splits
    random_state=42,
    n_jobs=-1           # Parallelisierung
)

model.fit(X_train, y_train)

# Evaluierung
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.2f}")


RMSE: 915.92
R²: 0.50


In [12]:
import joblib
joblib.dump(model, 'model_minergie.pkl')


['model_minergie.pkl']

In [1]:
import gradio as gr
import pandas as pd
import joblib

# 🔹 Daten laden
model = joblib.load("/workspaces/ai-applications-fs25/week2/test copy/fast_model_minergie.pkl")
apartments = pd.read_csv("/workspaces/ai-applications-fs25/week2/test copy/original_apartment_data_analytics_hs24_with_lat_lon.csv")
minergie_plz = pd.read_csv("/workspaces/ai-applications-fs25/week2/test copy/plz_häufigkeit.csv")

# 🔹 Sicherstellen, dass PLZ als Integer behandelt wird
apartments['postalcode'] = apartments['postalcode'].astype(int)
minergie_plz['PLZ'] = minergie_plz['PLZ'].astype(int)

# 🔹 Mapping von Ort zu PLZ
gemeinden_df = apartments[['postalcode', 'town']].drop_duplicates().sort_values('town')
gemeinde_options = dict(zip(gemeinden_df['town'], gemeinden_df['postalcode']))

beschreibung = """
🏡 **Wohnungspreis-Vorhersage mit Minergie-Feature**

🔹 Wähle **Wohnfläche, Zimmeranzahl & Gemeinde**  
🔹 Die App berechnet **zwei Preise**:
  - ✅ **Mit Minergie** (berücksichtigt energieeffiziente Gebäude)
  - ⚡ **Ohne Minergie** (keine Minergie-Gebäude berücksichtigt)

📊 Alle Daten basieren auf den hochgeladenen CSV-Dateien!
"""

# 🔹 Funktion zur Preisvorhersage
def predict_prices(area, rooms, ort):
    postalcode = gemeinde_options[ort]

    # Daten aus den geladenen CSV-Dateien abrufen
    coords = apartments.loc[apartments['postalcode'] == postalcode, ['lat', 'lon']].mean()
    minergie_count = minergie_plz.loc[minergie_plz['PLZ'] == postalcode, 'Häufigkeit'].values
    minergie_count = minergie_count[0] if len(minergie_count) else 0  # Falls kein Wert vorhanden ist

    # Modell-Eingaben vorbereiten
    input_mit_minergie = pd.DataFrame([{
        'area': area,
        'rooms': rooms,
        'lat': coords['lat'],
        'lon': coords['lon'],
        'minergie_anteil': minergie_count
    }])

    input_ohne_minergie = input_mit_minergie.copy()
    input_ohne_minergie['minergie_anteil'] = 0  # Minergie deaktivieren

    # Vorhersagen berechnen
    pred_mit = model.predict(input_mit_minergie)[0]
    pred_ohne = model.predict(input_ohne_minergie)[0]

    return f"✅ Preis mit Minergie: CHF {pred_mit:.2f}\n⚡ Preis ohne Minergie: CHF {pred_ohne:.2f}"

# 🔹 Gradio Interface
iface = gr.Interface(
    fn=predict_prices,
    inputs=[
        gr.Number(label="Wohnfläche (m²)"),
        gr.Number(label="Zimmeranzahl"),
        gr.Dropdown(choices=list(gemeinde_options.keys()), label="Ort")
    ],
    outputs="text",
    title="🏡 Wohnungspreis-Vorhersage (mit & ohne Minergie)",
    description=beschreibung
)

iface.launch()


/home/codespace/.python/current/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/home/codespace/.python/current/lib/python3.12/site-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/codespace/.python/current/lib/python3.12/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/codespace/.python/current/lib/python3.12/site-packages/gradio/blocks.py", line 2108, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/codespace/.python/current/lib/python3.12/site-packages/gradio/blocks.py", line 1655, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/codespace/.local/lib/python3.12/site-packages/anyio/to_thread.py", line 